In [29]:
import pandas as pd
import numpy as np
import re
import nltk
import json

- is there many licenses that have conflicing 
- create a "licenses_not_joined" file
 - collect statistics on how many licenses joined correctly with the adult-use / medicinal column.

In [30]:
def fix_nums(s):
    if pd.isnull(s):
        return np.nan
    else:
        return s.replace(".0", "")

### pick up where we left off. give mateo a list of already joined things. rank all of the stores.

In [32]:
left_to_join = json.load(open("..//data//latent.json", "r"))
left_to_join = [x[0] for x in list(filter(lambda x: x[1] == 0, {x: len(left_to_join[x]) for x in left_to_join}.items()))]

In [34]:
master_list = pd.read_csv("..//data/searchResultsRetailers.csv", low_memory = False)
master_list.zip_code = master_list.zip_code.fillna("").astype(str).apply(lambda x: fix_nums(x))
master_list.phone = master_list.phone.fillna("").astype(str).apply(lambda x: fix_nums(x))

weedmaps = pd.read_csv("..//data//store_clean.csv")

Remove hyphen from phone number in weedmaps:

In [35]:
def search_column(row, col, string):
    if pd.isnull(row[col]):
        return False
    elif re.search(string, row[col], flags = re.I) != None:
        return True
    else:
        return False

def update_matches(test, matches, num):
    
    for match in test.id:
        if match not in matches:
            matches.update({match: num})
        else:
            matches[match] += num
    
    return matches

In [36]:
def suggest_stores(master_list, weedmaps):
    
    tmp = 0
    ranked_indexes = {}
    unique_cities = weedmaps.city.unique()
    unique_phones = weedmaps.phone.unique()
    unique_website = [x.strip() for x in weedmaps.website.unique() if not pd.isnull(x)]
    unique_emails = [x.strip() for x in weedmaps.email.unique() if not pd.isnull(x)]
    
    for unjoined_store in left_to_join:
        
        comparison = master_list[master_list["License_no_dash"] == unjoined_store]
        matches = {}
        
        #print(comparison)
        #print(comparison.website.iloc[0])
        #print(comparison.email.iloc[0])
        #print(comparison.phone.iloc[0])
        #print(comparison.zip_code.iloc[0])
        #print(comparison.city.iloc[0])
        #print(comparison.company_name.iloc[0])

        #print()
        # compare website
        if not pd.isnull(comparison.website.iloc[0]):
            best_site = sorted({x: nltk.edit_distance(comparison.website.iloc[0], str(x)) for x in unique_website}.items(), key = lambda x: x[1])[0]
            #print(best_site)
            if best_site[1] <= 2:
                test = weedmaps[weedmaps.apply(lambda row: search_column(row, "website", best_site[0]), axis = 1)]
                matches = update_matches(test, matches, 3)
        
        
        # compare email
        if not pd.isnull(comparison.email.iloc[0]):
            best_email = sorted({x: nltk.edit_distance(comparison.email.iloc[0], str(x)) for x in unique_emails}.items(), key = lambda x: x[1])[0]
            #print(best_email)
            if best_email[1] <= 2:
                test = weedmaps[weedmaps.apply(lambda row: search_column(row, "email", best_email[0]), axis = 1)]
                matches = update_matches(test, matches, 3)
        
        
        # compare phone
        if not pd.isnull(comparison.phone.iloc[0]):
            best_phone = sorted({x: nltk.edit_distance(comparison.phone.iloc[0], x) for x in unique_phones}.items(), key = lambda x: x[1])[0]
            #print(best_phone)
            if best_phone[1] <= 1:
                test = weedmaps[weedmaps.apply(lambda row: search_column(row, "phone", best_phone[0]), axis = 1)]
                matches = update_matches(test, matches, 3)
        
        
        # compare city
        if not pd.isnull(comparison.city.iloc[0]):
            best_city = sorted({x: nltk.edit_distance(comparison.city.iloc[0], x) for x in unique_cities}.items(), key = lambda x: x[1])[0][0]
            #print(best_city)
            test = weedmaps[weedmaps.apply(lambda row: search_column(row, "city", best_city), axis = 1)]    
            matches = update_matches(test, matches, 1)
            
            
        # compare names
        if not pd.isnull(comparison.company_name.iloc[0]):
            best_name = sorted({x: nltk.edit_distance(comparison.company_name.iloc[0], x) for x in unique_cities}.items(), key = lambda x: x[1])[0][0]
            #print(best_name)
            test = weedmaps[weedmaps.apply(lambda row: search_column(row, "name", best_city), axis = 1)]
            matches = update_matches(test, matches, 1)  
        
        
        # compare zip
        test = weedmaps[weedmaps.apply(lambda row: search_column(row, "zip_code", comparison.zip_code.iloc[0]), axis = 1)]
        #print(test)
        matches = update_matches(test, matches, 1)
        
        ranked_indexes.update({unjoined_store: sorted(matches.items(), key = lambda x: x[1], reverse = True)})
        
    return ranked_indexes

In [37]:
all_matches = suggest_stores(master_list, weedmaps)

In [38]:
with open("..//data//matches.json", "w") as f:
    json.dump(all_matches, f)

.

.

.

.

.

.

.

.

.

.

.

.

.

## match email

In [5]:
left_to_merge.head()

,Unnamed: 0,Unnamed: 0.1,id,name,state,city,latitude,longitude,license_type,address,...,retailer_services,phone,adult_use_cultivation,adult_use_nonstorefront,adult_use_retail,distributor,medical_cultivation,medical_nonstorefront,medical_retail,microbusiness
0,11,11,50268,Lytt,CA,Pacifica,37.634507,-122.491800,medical,1905 Palmetto Ave. Unit B,...,storefront,0000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,14,60567,Harvest of Napa (Newly Opened),CA,Napa,38.297243,-122.301418,medical,2449 Second Street,...,storefront,707.210.2271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,15,49957,Mendocino Organics,CA,Vallejo,38.109695,-122.253425,medical,539 A Tennessee ST,...,storefront,(707)652-5018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,16,20208,Fly Fast Delivery - Richmond,CA,Richmond,37.964331,-122.318325,medical,NaN,...,delivery,(510) 776-2965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,22,57685,East Bay Therapeutics,CA,Emeryville,37.838307,-122.290292,hybrid,5745 Peladeau Street,...,storefront,(510) 925-1266,NaN,NaN,A100000368,NaN,NaN,NaN,M100000420,NaN


In [93]:
master_list[master_list.apply(lambda row: search_column(row, "Business Contact Information", "vhhc420@gmail.com"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
1869,C11-18-0000083-TEMP,Cannabis - Distributor Temporary License,Greogry Schoepp,"VHHC, LLC : Email- vhhc420@gmail.com : Phone- ...",Limited Liability Company,"VALLEJO, CA 945904430 County: SOLANO",Active,06/11/2019,08/15/2018,N/A for this license type,BOTH,94590


## match phone

In [90]:
master_list[master_list.apply(lambda row: search_column(row, "Business Contact Information", "7076525018"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
1869,C11-18-0000083-TEMP,Cannabis - Distributor Temporary License,Greogry Schoepp,"VHHC, LLC : Email- vhhc420@gmail.com : Phone- ...",Limited Liability Company,"VALLEJO, CA 945904430 County: SOLANO",Active,06/11/2019,08/15/2018,N/A for this license type,BOTH,94590


## match zip

In [72]:
master_list[master_list.apply(lambda row: search_column(row, "zip_code", "94903"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
1544,C9-18-0000070-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Alexa Wall,"LUMALIGN, LLC : Moonflower : Email- alexa@moo...",Limited Liability Company,"SAN RAFAEL, CA 949031700 County: MARIN",Active,09/15/2019,11/19/2018,N/A for this license type,Medicinal,94903


## match city

In [74]:
master_list[master_list.apply(lambda row: search_column(row, "Premise Address", "Marin"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
154,C13-18-0000156-TEMP,Cannabis - Distributor-Transport Only Temporar...,Albert da Cunha,IBALD Enterprises : Marin Medicinals : Email-...,Limited Liability Company,"SAN RAFAEL, CA 94901 County: MARIN",Active,07/27/2019,12/29/2018,N/A for this license type,Medicinal,94901
742,C13-18-0000113-TEMP,Cannabis - Distributor-Transport Only Temporar...,Joaquin Lujan,Tamalpais Co-Packing : Email- joaquintlujan@gm...,Limited Liability Company,"SAN RAFAEL, CA 949014720 County: MARIN",Active,07/18/2019,12/20/2018,N/A for this license type,Medicinal,94901
769,C11-18-0000603-TEMP,Cannabis - Distributor Temporary License,Monica Barker,"NICE GUYS DISTRIBUTION, INC. : Email- monica@n...",Corporation,"SAN RAFAEL, CA 949015374 County: MARIN",Active,07/17/2019,12/19/2018,N/A for this license type,Medicinal,94901
1184,C9-18-0000104-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Nourit Rafael,BUTTERCUP & SPRING : ONA.life : Email- Nurit@...,Corporation,"SAN RAFAEL, CA 94901 County: MARIN",Active,07/08/2019,12/10/2018,N/A for this license type,Medicinal,94901
1544,C9-18-0000070-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Alexa Wall,"LUMALIGN, LLC : Moonflower : Email- alexa@moo...",Limited Liability Company,"SAN RAFAEL, CA 949031700 County: MARIN",Active,09/15/2019,11/19/2018,N/A for this license type,Medicinal,94903
1741,C9-18-0000043-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Emily Elam,FIREFLY MARIN : Firefly Marin : Email- emily....,Limited Liability Company,"San Rafael, CA 94901 County: MARIN",Active,08/01/2019,10/05/2018,N/A for this license type,Medicinal,94901
1808,C11-18-0000102-TEMP,Cannabis - Distributor Temporary License,George Bianchini,LIPOSOME FORMULATIONS .INC : Email- niaid@well...,Corporation,"NOVATO, CA 94949 County: MARIN",Active,07/02/2019,09/05/2018,N/A for this license type,Medicinal,94949
2043,C9-18-0000005-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Robert Elam,ACCESS MARIN : California Wellness : Email- e...,Limited Liability Company,"SAN RAFAEL, CA 949015571 County: MARIN",Canceled,10/11/2018,06/13/2018,N/A for this license type,Medicinal,94901
2108,M9-18-0000105-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Brian Bjork,THE BJORK GROUP : MARIN GARDENS : Email- bria...,Corporation,"SAN RAFAEL, CA 94901 County: MARIN",Active,06/16/2019,05/22/2018,N/A for this license type,BOTH,94901
2280,M9-18-0000091-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Scott Perkins,"CW GROUP, INC. : California Wellness : Email-...",Corporation,"SAN RAFAEL, CA 949015571 County: MARIN",Expired,08/11/2018,04/13/2018,N/A for this license type,Medicinal,94901


In [37]:
def suggest_stores(left_to_merge, master_list):
    
    tmp = 0
    ranked_indexes = {}
    unique_cities = master_list.city.unique()

    for index in left_to_merge.id:
        comparison = left_to_merge[left_to_merge.id == index]
        matches = {}

        # check phone
        #print(ranked_indexes)
        print(comparison.phone.iloc[0])
        print(comparison.zip_code.iloc[0])
        print(comparison.city.iloc[0])
        #print()
        
        test = master_list[master_list.apply(lambda row: search_column(row, "Business Contact Information", comparison.phone.iloc[0]), axis = 1)]    
        matches = update_matches(test, matches, 2)

        # check zip
        test = master_list[master_list.apply(lambda row: search_column(row, "zip_code", comparison.zip_code.iloc[0]), axis = 1)]    
        matches = update_matches(test, matches, 1)

        # check city
        # first find closest city wrt levenschtein distance
        best_city = sorted({x: nltk.edit_distance(comparison.city.iloc[0].lower(), x) for x in unique_cities}.items(), key = lambda x: x[1])[0][0]
        #print(best_city)
        # then find those indices
        test = master_list[master_list.apply(lambda row: search_column(row, "city", best_city), axis = 1)]    
        matches = update_matches(test, matches, 1)

        # sort matches
        ranked_indexes.update({index: sorted(matches.items(), key = lambda x: x[1], reverse = True)})
        
        tmp += 1
        if tmp > 20:
            break

    return ranked_indexes    